# ConversationSummaryMemory

이제 조금 더 복잡한 메모리 유형인 `ConversationSummaryMemory` 를 사용하는 방법을 살펴 보겠습니다.

이 유형의 메모리는 시간 경과에 따른 **대화의 요약** 을 생성합니다. 이는 시간 경과에 따른 대화의 정보를 압축하는 데 유용할 수 있습니다.

대화 요약 메모리는 대화가 진행되는 동안 대화를 요약하고 **현재 요약을 메모리에 저장** 합니다.

그런 다음 이 메모리를 사용하여 지금까지의 대화 요약을 프롬프트/체인에 삽입할 수 있습니다.

이 메모리는 과거 메시지 기록을 프롬프트에 그대로 보관하면 토큰을 너무 많이 차지할 수 있는 긴 대화에 가장 유용합니다.


`ConversationSummaryMemory` 를 생성합니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate


# 1) summarizer
summarizer = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

def summarize_messages(summary, new_message):
    prompt = f"""
    Here is the conversation summary so far:
    {summary}

    The user said:
    {new_message}

    Update the summary concisely.
    """
    result = summarizer.invoke(prompt)
    return result.content


# 2) history & summary
history = InMemoryChatMessageHistory()
summary_state = {"summary": ""}


# 3) main LLM
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)


# 4) prepare_input: 요약 갱신 + 평탄화된 변수 리턴
def prepare_input(input_dict):
    user_msg = input_dict["input"]

    new_summary = summarize_messages(summary_state["summary"], user_msg)
    summary_state["summary"] = new_summary

    return {
        "history_summary": new_summary,
        "user_input": user_msg
    }


# prepare_input을 Runnable로 래핑 (assign() 대신 이걸 사용해야 함)
prepare_chain = RunnableLambda(prepare_input)


# 5) 프롬프트 템플릿
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful travel assistant. Use the conversation summary to answer questions."),
    ("system", "Conversation Summary:\n{history_summary}"),
    ("human", "{user_input}")
])


# 6) 단 한 번만 LLM 호출되는 체인
chat_chain = prepare_chain | prompt | llm


# 7) RunnableWithMessageHistory로 세션 관리
conversation = RunnableWithMessageHistory(
    chat_chain,
    lambda session_id: history,
    input_messages_key="input",
    history_messages_key="history"
)


여러 대화를 저장하도록 합니다.


In [3]:
# memory.save_context(
#     inputs={"human": "유럽 여행 패키지의 가격은 얼마인가요?"},
#     outputs={
#         "ai": "유럽 14박 15일 패키지의 기본 가격은 3,500유로입니다. 이 가격에는 항공료, 호텔 숙박비, 지정된 관광지 입장료가 포함되어 있습니다. 추가 비용은 선택하신 옵션 투어나 개인 경비에 따라 달라집니다."
#     },
# )
# memory.save_context(
#     inputs={"human": "여행 중에 방문할 주요 관광지는 어디인가요?"},
#     outputs={
#         "ai": "이 여행에서는 파리의 에펠탑, 로마의 콜로세움, 베를린의 브란덴부르크 문, 취리히의 라이네폴 등 유럽의 유명한 관광지들을 방문합니다. 각 도시의 대표적인 명소들을 포괄적으로 경험하실 수 있습니다."
#     },
# )
# memory.save_context(
#     inputs={"human": "여행자 보험은 포함되어 있나요?"},
#     outputs={
#         "ai": "네, 모든 여행자에게 기본 여행자 보험을 제공합니다. 이 보험은 의료비 지원, 긴급 상황 발생 시 지원 등을 포함합니다. 추가적인 보험 보장을 원하시면 상향 조정이 가능합니다."
#     },
# )
# memory.save_context(
#     inputs={
#         "human": "항공편 좌석을 비즈니스 클래스로 업그레이드할 수 있나요? 비용은 어떻게 되나요?"
#     },
#     outputs={
#         "ai": "항공편 좌석을 비즈니스 클래스로 업그레이드하는 것이 가능합니다. 업그레이드 비용은 왕복 기준으로 약 1,200유로 추가됩니다. 비즈니스 클래스에서는 더 넓은 좌석, 우수한 기내식, 그리고 추가 수하물 허용량 등의 혜택을 제공합니다."
#     },
# )
# memory.save_context(
#     inputs={"human": "패키지에 포함된 호텔의 등급은 어떻게 되나요?"},
#     outputs={
#         "ai": "이 패키지에는 4성급 호텔 숙박이 포함되어 있습니다. 각 호텔은 편안함과 편의성을 제공하며, 중심지에 위치해 관광지와의 접근성이 좋습니다. 모든 호텔은 우수한 서비스와 편의 시설을 갖추고 있습니다."
#     },
# )
# memory.save_context(
#     inputs={"human": "식사 옵션에 대해 더 자세히 알려주실 수 있나요?"},
#     outputs={
#         "ai": "이 여행 패키지는 매일 아침 호텔에서 제공되는 조식을 포함하고 있습니다. 점심과 저녁 식사는 포함되어 있지 않아, 여행자가 자유롭게 현지의 다양한 음식을 경험할 수 있는 기회를 제공합니다. 또한, 각 도시별로 추천 식당 리스트를 제공하여 현지의 맛을 최대한 즐길 수 있도록 도와드립니다."
#     },
# )
# memory.save_context(
#     inputs={"human": "패키지 예약 시 예약금은 얼마인가요? 취소 정책은 어떻게 되나요?"},
#     outputs={
#         "ai": "패키지 예약 시 500유로의 예약금이 필요합니다. 취소 정책은 예약일로부터 30일 전까지는 전액 환불이 가능하며, 이후 취소 시에는 예약금이 환불되지 않습니다. 여행 시작일로부터 14일 전 취소 시 50%의 비용이 청구되며, 그 이후는 전액 비용이 청구됩니다."
#     },
# )

history.add_user_message("유럽 여행 패키지의 가격은 얼마인가요?")
history.add_ai_message(
    "유럽 14박 15일 패키지의 기본 가격은 3,500유로입니다. 이 가격에는 항공료, 호텔 숙박비, 지정된 관광지 입장료가 포함되어 있습니다. 추가 비용은 선택하신 옵션 투어나 개인 경비에 따라 달라집니다."
)

history.add_user_message("여행 중에 방문할 주요 관광지는 어디인가요?")
history.add_ai_message(
    "이 여행에서는 파리의 에펠탑, 로마의 콜로세움, 베를린의 브란덴부르크 문, 취리히의 라이네폴 등 유럽의 유명한 관광지들을 방문합니다. 각 도시의 대표적인 명소들을 포괄적으로 경험하실 수 있습니다."
)

history.add_user_message("여행자 보험은 포함되어 있나요?")
history.add_ai_message(
    "네, 모든 여행자에게 기본 여행자 보험을 제공합니다. 이 보험은 의료비 지원, 긴급 상황 발생 시 지원 등을 포함합니다. 추가적인 보험 보장을 원하시면 상향 조정이 가능합니다."
)

history.add_user_message("항공편 좌석을 비즈니스 클래스로 업그레이드할 수 있나요? 비용은 어떻게 되나요?")
history.add_ai_message(
    "항공편 좌석을 비즈니스 클래스로 업그레이드하는 것이 가능합니다. 업그레이드 비용은 왕복 기준으로 약 1,200유로 추가됩니다. 비즈니스 클래스에서는 더 넓은 좌석, 우수한 기내식, 그리고 추가 수하물 허용량 등의 혜택을 제공합니다."
)

history.add_user_message("패키지에 포함된 호텔의 등급은 어떻게 되나요?")
history.add_ai_message(
    "이 패키지에는 4성급 호텔 숙박이 포함되어 있습니다. 각 호텔은 편안함과 편의성을 제공하며, 중심지에 위치해 관광지와의 접근성이 좋습니다. 모든 호텔은 우수한 서비스와 편의 시설을 갖추고 있습니다."
)

history.add_user_message("식사 옵션에 대해 더 자세히 알려주실 수 있나요?")
history.add_ai_message(
    "이 여행 패키지는 매일 아침 호텔에서 제공되는 조식을 포함하고 있습니다. 점심과 저녁 식사는 포함되어 있지 않아, 여행자가 자유롭게 현지의 다양한 음식을 경험할 수 있는 기회를 제공합니다. 또한, 각 도시별로 추천 식당 리스트를 제공하여 현지의 맛을 최대한 즐길 수 있도록 도와드립니다."
)

history.add_user_message("패키지 예약 시 예약금은 얼마인가요? 취소 정책은 어떻게 되나요?")
history.add_ai_message(
    "패키지 예약 시 500유로의 예약금이 필요합니다. 취소 정책은 예약일로부터 30일 전까지는 전액 환불이 가능하며, 이후 취소 시에는 예약금이 환불되지 않습니다. 여행 시작일로부터 14일 전 취소 시 50%의 비용이 청구되며, 그 이후는 전액 비용이 청구됩니다."
)

저장된 메모리의 history 를 확인합니다.

이전의 모든 대화를 압축적으로 요약한 내용을 확인할 수 있습니다.


In [5]:
# 저장된 메모리 확인

print("현재 요약 상태:")
print(summary_state["summary"])


response = conversation.invoke(
    {"input": "패키지의 전체 요약을 알려주세요."},
    config={"configurable": {"session_id": "session1"}}
)
print("\nLLM 응답:")
print(response.content)

현재 요약 상태:
The user asked for an overall summary of the package.

LLM 응답:
이 패키지는 사용자가 요청한 기능이나 정보를 제공하는 데 필요한 주요 요소와 기능들을 포함하고 있으며, 사용자의 요구에 맞춘 효율적이고 체계적인 구성을 갖추고 있습니다.


## ConversationSummaryBufferMemory


`ConversationSummaryBufferMemory` 는 두 가지 아이디어를 결합한 것입니다.

최근 대화내용의 버퍼를 메모리에 유지하되, 이전 대화내용을 완전히 플러시(flush)하지 않고 요약으로 컴파일하여 두 가지를 모두 사용합니다.

대화내용을 플러시할 시기를 결정하기 위해 상호작용의 개수가 아닌 **토큰 길이** 를 사용합니다.


In [6]:
# from langchain.memory import ConversationTokenBufferMemory
# from langchain_openai import ChatOpenAI


# # LLM 모델 생성
# llm = ChatOpenAI(model_name="gpt-4.1-mini")

# # 메모리 설정
# memory = ConversationTokenBufferMemory(
#     llm=llm, max_token_limit=150, return_messages=True  # 최대 토큰 길이를 50개로 제한
# )

import tiktoken
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

# 1. LLM
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 2. Tokenizer (OpenAI 계열 모델에 맞는 기본 토크나이저)
enc = tiktoken.encoding_for_model("gpt-4.1-mini")

# 3. 토큰 기반 메모리 설정

history = InMemoryChatMessageHistory()

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="messages"),
])

chain = prompt | llm

MAX_TOKEN_LIMIT = 150  # 최대 토큰 수 설정

def summarize_messages(messages):
    #오래된 메시지들을 하나의 요약 메시지로 압축
    text = ""
    for m in messages:
        role = "Human" if isinstance(m, HumanMessage) else "AI"
        text += f"{role}: {m.content}\n"

    summary_prompt = f"다음 대화를 1개의 간단한 요약으로 만들어줘:\n{text}"
    summary = llm.invoke(summary_prompt)
    return AIMessage(content=f"[요약된 이전 대화]\n{summary.content}")

def count_tokens(messages):
    #전체 메시지를 단일 텍스트로 연결해서 토큰 수 계산
    text = ""
    for m in messages:
        text += m.content + "\n"
    return len(enc.encode(text))

def add_message_with_token_limit(history_obj, message):
    #토큰 수가 MAX_TOKEN_LIMIT를 넘으면 압축
    history_obj.add_message(message)

    while count_tokens(history_obj.messages) > MAX_TOKEN_LIMIT:

        # 1) 오래된 절반 메시지들을 잘라 요약으로 만든다
        cutoff = len(history_obj.messages) // 2
        old_msgs = history_obj.messages[:cutoff]

        summary_msg = summarize_messages(old_msgs)

        # 2) 기존 메시지에서 오래된 부분 제거
        history_obj.messages = history_obj.messages[cutoff:]

        # 3) 요약 메시지를 맨 앞에 삽입
        history_obj.messages.insert(0, summary_msg)


conversation = RunnableWithMessageHistory(
    chain,
    lambda session_id: history,
    input_messages_key="messages",
    history_messages_key="messages"
)

먼저, 1개의 대화만 저장해 보도록 한 뒤, 메모리를 확인해 보겠습니다.


In [7]:
# memory.save_context(
#     inputs={"human": "유럽 여행 패키지의 가격은 얼마인가요?"},
#     outputs={
#         "ai": "유럽 14박 15일 패키지의 기본 가격은 3,500유로입니다. 이 가격에는 항공료, 호텔 숙박비, 지정된 관광지 입장료가 포함되어 있습니다. 추가 비용은 선택하신 옵션 투어나 개인 경비에 따라 달라집니다."
#     },
# )

add_message_with_token_limit(
    history,
    HumanMessage(content="유럽 여행 패키지의 가격은 얼마인가요?")
)

add_message_with_token_limit(
    history,
    AIMessage(content="유럽 14박 15일 패키지의 기본 가격은 3,500유로입니다. "
                      "이 가격에는 항공료, 호텔 숙박비, 지정된 관광지 입장료가 포함되어 있습니다. "
                      "추가 비용은 선택하신 옵션 투어나 개인 경비에 따라 달라집니다.")
)


print("현재 요약 상태:")
print(conversation.invoke(
    {"messages": []},
    config={"configurable": {"session_id": "session1"}}
).content)


현재 요약 상태:
유럽 여행 패키지의 가격은 여행 기간, 포함되는 서비스, 여행사에 따라 다르지만, 일반적으로 7박 8일 기준으로 150만 원에서 300만 원 사이입니다. 좀 더 구체적인 정보를 원하시면 여행 일정이나 포함 사항을 알려주시면 자세히 안내해 드리겠습니다.


메모리에 저장된 대화를 확인합니다.

아직은 대화내용을 요약하지 않습니다. 기준이 되는 **200** 토큰에 도달하지 않았기 때문입니다.


In [8]:
# 메모리에 저장된 대화내용 확인
for msg in history.messages:
    role = "User" if isinstance(msg, HumanMessage) else "AI"
    print(f"{role}: {msg.content}")

User: 유럽 여행 패키지의 가격은 얼마인가요?
AI: 유럽 14박 15일 패키지의 기본 가격은 3,500유로입니다. 이 가격에는 항공료, 호텔 숙박비, 지정된 관광지 입장료가 포함되어 있습니다. 추가 비용은 선택하신 옵션 투어나 개인 경비에 따라 달라집니다.
User: 유럽 여행 패키지의 가격은 얼마인가요?
AI: 유럽 14박 15일 패키지의 기본 가격은 3,500유로입니다. 이 가격에는 항공료, 호텔 숙박비, 지정된 관광지 입장료가 포함되어 있습니다. 추가 비용은 선택하신 옵션 투어나 개인 경비에 따라 달라집니다.
AI: 유럽 여행 패키지의 가격은 여행 기간, 포함되는 서비스, 여행사에 따라 다르지만, 일반적으로 7박 8일 기준으로 150만 원에서 300만 원 사이입니다. 좀 더 구체적인 정보를 원하시면 여행 일정이나 포함 사항을 알려주시면 자세히 안내해 드리겠습니다.


대화를 추가로 저장하여 200 토큰 제한을 넘기도록 해 보겠습니다.


In [9]:
add_message_with_token_limit(
    history,
    HumanMessage(content="여행 중에 방문할 주요 관광지는 어디인가요?")
)
add_message_with_token_limit(
    history,
    AIMessage(content="이 여행에서는 파리의 에펠탑, 로마의 콜로세움, 베를린의 브란덴부르크 문, 취리히의 라이네폴 등 유럽의 유명한 관광지들을 방문합니다. 각 도시의 대표적인 명소들을 포괄적으로 경험하실 수 있습니다.")
)

add_message_with_token_limit(
    history,
    HumanMessage(content="여행자 보험은 포함되어 있나요?")
)
add_message_with_token_limit(
    history,
    AIMessage(content="네, 모든 여행자에게 기본 여행자 보험을 제공합니다. 이 보험은 의료비 지원, 긴급 상황 발생 시 지원 등을 포함합니다. 추가적인 보험 보장을 원하시면 상향 조정이 가능합니다.")
)

add_message_with_token_limit(
    history,
    HumanMessage(content="항공편 좌석을 비즈니스 클래스로 업그레이드할 수 있나요? 비용은 어떻게 되나요?")
)
add_message_with_token_limit(
    history,
    AIMessage(content="항공편 좌석을 비즈니스 클래스로 업그레이드하는 것이 가능합니다. 업그레이드 비용은 왕복 기준으로 약 1,200유로 추가됩니다. 비즈니스 클래스에서는 더 넓은 좌석, 우수한 기내식, 그리고 추가 수하물 허용량 등의 혜택을 제공합니다.")
)

add_message_with_token_limit(
    history,
    HumanMessage(content="패키지에 포함된 호텔의 등급은 어떻게 되나요?")
)
add_message_with_token_limit(
    history,
    AIMessage(content="이 패키지에는 4성급 호텔 숙박이 포함되어 있습니다. 각 호텔은 편안함과 편의성을 제공하며, 중심지에 위치해 관광지와의 접근성이 좋습니다. 모든 호텔은 우수한 서비스와 편의 시설을 갖추고 있습니다.")
)


저장된 대화내용을 확인합니다. 가장 최근 1개의 대화에 대해서는 요약이 진행되지 않지만, 이전의 대화내용은 요약본으로 저장되어 있습니다.


In [10]:
# 메모리에 저장된 대화내용 확인
print("현재 메모리 상태:")
print(history.messages)

현재 메모리 상태:
[AIMessage(content='[요약된 이전 대화]\n유럽 여행 패키지의 가격, 일정, 기본 여행자 보험 포함 여부와 함께 항공편 비즈니스 클래스 업그레이드 가능성과 비용(약 1,200유로 추가)에 대해 문의했다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='패키지에 포함된 호텔의 등급은 어떻게 되나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='이 패키지에는 4성급 호텔 숙박이 포함되어 있습니다. 각 호텔은 편안함과 편의성을 제공하며, 중심지에 위치해 관광지와의 접근성이 좋습니다. 모든 호텔은 우수한 서비스와 편의 시설을 갖추고 있습니다.', additional_kwargs={}, response_metadata={})]
